## Start Python


In [ ]:
import os
print("PYTHONPATH:", os.environ.get('PYTHONPATH'))
print("PATH:", os.environ.get('PATH'))

## Calcurate delta V needed


In [ ]:
import pykep as pk
import numpy as np
import matplotlib.pyplot as plt # 描画ライブラリ

GM = 398600.4354360959 # 地球の重力定数, km3/s2
initOrbit = 400 # km
targetOrbit = 35786 #km
re = pk.EARTH_RADIUS / 1000 # km
a1 = re + initOrbit
a2 = re + targetOrbit

r1, v1 = pk.par2ic([
                    a1, # a, km
                    0.0, # e, -
                    0.0, # i, rad
                    0.0, # O, rad
                    0.0, # w, rad
                    0.0 # E, rad
                 ], GM)
print(r1, v1)

r2, v2 = pk.par2ic([
                    a2, # a, km
                    0.0, # e, -
                    0.0, # i, rad
                    0.0, # O, rad
                    0.0, # w, rad
                    0.0 # E, rad
                 ], GM)
print(r2, v2)

ea_all = np.linspace(0, 2*np.pi, 1000) # 0 to 2pi radian


# 軌道伝搬
pos1 = np.array([ 
        pk.par2ic([
                    a1, # a, km
                    0.0, # e, -
                    0.0, # i, rad
                    0.0, # O, rad
                    0.0, # w, rad
                    ea_t # E, rad
                   ], GM)[0] # 位置のみを取り出す
        for ea_t in ea_all
      ]) # Pythonのリスト内包表記を用いて，一括で計算

pos2 = np.array([ 
        pk.par2ic([
                    a2, # a, km
                    0.0, # e, -
                    0.0, # i, rad
                    0.0, # O, rad
                    0.0, # w, rad
                    ea_t # E, rad
                   ], GM)[0] # 位置のみを取り出す
        for ea_t in ea_all
      ]) # Pythonのリスト内包表記を用いて，一括で計算

tof = 19048.4026989538 # GTO半周分の時間, s


# Lambert問題を解く

r1 = (-r1[0], 0.0001, r1[2])

l = pk.lambert_problem(r1 = r1, r2 = r2, tof = tof, mu = GM)

# ΔVの計算
v_LEO = np.sqrt(GM/6778.0) # LEOの速度, km/s
v_GEO = np.sqrt(GM/42164.0) # GEOの速度, km/s
v1 = l.get_v1()[0] # Lambert問題から計算された初期の速度
v2 = l.get_v2()[0] # Lambert問題から計算された終端の速度
dv1 = abs(v1[0]) - v_LEO
dv2 = v_GEO - abs(v2[0])

print(dv1, dv2)

# Lambert問題を解く
r1 = np.array([0.01, -a1, 0.0]) # LEO上の位置, km
r2 = np.array([0.01, a2, 0.0]) # GEO上の位置, km

l = pk.lambert_problem(r1 = r1, r2 = r2, tof = tof, mu = GM)

# ΔVの計算
v_LEO = np.sqrt(GM/6778.0) # LEOの速度, km/s
v_GEO = np.sqrt(GM/42164.0) # GEOの速度, km/s
v1 = l.get_v1()[0] # Lambert問題から計算された初期の速度
v2 = l.get_v2()[0] # Lambert問題から計算された終端の速度
dv1 = abs(v1[0]) - v_LEO
dv2 = v_GEO - abs(v2[0])

print(dv1, dv2)

# 描画
plt.close() 
plt.plot(pos1[:,0],pos1[:,1], 'b')
plt.plot(pos2[:,0],pos2[:,1], 'g')

plt.grid() # 格子をつける
plt.gca().set_aspect('equal') # グラフのアスペクト比を揃える
plt.show()

In [ ]:
# We need to import some toolboxes
import pykep as pk
from pykep.planet import keplerian, jpl_lp
from pykep import AU, MU_SUN, epoch, DAY2SEC, SEC2DAY, RAD2DEG, DEG2RAD


initOrbit = 400 # km
targetOrbit = 35786 #km

# LEOとGEOの軌道要素を定義
leo = keplerian(epoch(0), [pk.EARTH_RADIUS + initOrbit, 0, 0, 0, 0, 0], pk.MU_EARTH)
geo = keplerian(epoch(0), [pk.EARTH_RADIUS + targetOrbit, 0, 0, 0, 0, 0], pk.MU_EARTH)

# ホーマン遷移のΔVを計算
dv1, dv2 = pk.lambert_problem(leo, geo, 0.5 * pk.DAY2SEC, pk.MU_EARTH).get_dvs()

# 結果の表示
print("ΔV for Hohmann transfer from LEO to GEO:")
print("First burn ΔV: ", dv1)
print("Second burn ΔV: ", dv2)
